# 📈 Interest Rate Futures

## Introduction
This notebook provides a comprehensive exploration of **interest rate futures** as outlined in Chapter 6 of John Hull's *Options, Futures, and Other Derivatives*. This chapter covers the fundamental concepts, pricing mechanisms, and applications of **interest rate futures contracts** in managing and hedging interest rate risk.

## 💼 Interest Rate Futures Contracts
An **interest rate futures contract** is a standardized agreement to buy or sell a specified amount of a financial instrument or asset at a future date, with the price set today. These contracts are primarily used for hedging or speculating on future movements in interest rates. Common interest rate futures include those based on short-term instruments (like Eurodollar and Treasury bill futures) and longer-term instruments (like Treasury bond futures).

### 📊 Key Elements of Interest Rate Futures
- **Underlying Asset**: The asset may be a government bond, a Treasury bill, or a Eurodollar deposit, which determines the contract's interest rate sensitivity.
  
- **Contract Terms**: Each futures contract has standardized terms regarding the delivery date, contract size, and pricing method, making it highly liquid and attractive to both hedgers and speculators.

### 💡 Pricing and Valuation
The value of an interest rate futures contract depends on the movement of underlying interest rates. Key pricing approaches include:

- **Marking to Market**: Each futures contract is marked to market daily, adjusting for gains and losses based on the settlement price.
  
- **Conversion Factors**: For bond futures, a conversion factor is used to adjust for differences in coupon rates and maturities among deliverable bonds.

## 🔍 Applications of Interest Rate Futures
Interest rate futures are versatile instruments widely used by financial institutions, corporations, and investors. Their applications include:

- **Hedging**: Financial institutions use interest rate futures to protect against fluctuations in interest rates that may impact asset or liability portfolios.
  
- **Speculation**: Traders may use these contracts to bet on future changes in interest rates, capitalizing on expectations of rising or falling rates.
  
- **Managing Duration and Portfolio Sensitivity**: Interest rate futures provide an efficient way to adjust a portfolio’s sensitivity to interest rate movements without directly buying or selling bonds.


In [10]:
import pandas as pd
import numpy as np

# ***Day Count Conventions***

In [11]:
# Let's take a bond with semestrial coupons payments.
start_semester = "2020-03-01"
end_semester = "2020-09-01"
coupon = 4
date = "2020-07-03" # We want to compute the accrued interest on the bond on 2020-07-03.

# ======= Actual/Actual (ISDA) =======
total_period = (pd.to_datetime(end_semester) - pd.to_datetime(start_semester)).days
accrued_period = (pd.to_datetime(date) - pd.to_datetime(start_semester)).days
print(f'The semester is {total_period} days and the accrued period is {accrued_period} days.')

accrued_interest = (accrued_period / total_period) * coupon
print(f'The accrued interest is {accrued_interest}.')

The semester is 184 days and the accrued period is 124 days.
The accrued interest is 2.6956521739130435.


In [12]:
# ======= 30/360 =======
#? As python doesn't provide a built-in function to compute the 30/360 day count convention, we need to implement it.
def days_30_360(start_date, end_date):
    """
    Python implementation of the 30/360 day count convention.
    
    Args:
        start_date (str): The start date.
        end_date (str): The end date.
    
    Returns:
        int: The number of days between the two dates according
    """
    # ======= I. Convert the dates to datetime objects =======
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # ======= II. Adjust the day component =======
    # The day component of each date is adjusted so that any day greater than 30 is set to 30
    start_day = min(30, start_date.day)
    end_day = min(30, end_date.day)

    # ======= III. Compute the difference in days =======
    year_diff = end_date.year - start_date.year
    month_diff = end_date.month - start_date.month
    day_diff = end_day - start_day

    return 360 * year_diff + 30 * month_diff + day_diff

total_period = days_30_360(start_semester, end_semester)
accrued_period = days_30_360(start_semester, date)
print(f'The semester is {total_period} days and the accrued period is {accrued_period} days.')

accrued_interest = (accrued_period / total_period) * coupon
print(f'The accrued interest is {accrued_interest}.')

The semester is 180 days and the accrued period is 122 days.
The accrued interest is 2.7111111111111112.


In [13]:
"""
Let's compare the two day count conventions with the infamous February 28th.
"""
start_semester = "2018-02-28"
end_semester = "2018-07-01"
coupon = 4
date = "2018-03-01" 

# ======= Actual/Actual (ISDA) =======
total_period = (pd.to_datetime(end_semester) - pd.to_datetime(start_semester)).days
accrued_period = (pd.to_datetime(date) - pd.to_datetime(start_semester)).days
print(f'The semester is {total_period} days and the accrued period is {accrued_period} days.')

accrued_interest = (accrued_period / total_period) * coupon
print(f'The accrued interest is {accrued_interest}.')

print("===============================================")
# ======= 30/360 =======
total_period = days_30_360(start_semester, end_semester)
accrued_period = days_30_360(start_semester, date)
print(f'The semester is {total_period} days and the accrued period is {accrued_period} days.')

accrued_interest = (accrued_period / total_period) * coupon
print(f'The accrued interest is {accrued_interest}.')

The semester is 123 days and the accrued period is 1 days.
The accrued interest is 0.032520325203252036.
The semester is 123 days and the accrued period is 3 days.
The accrued interest is 0.0975609756097561.


# ***Listing of Bonds***

In [14]:
"""
Example of a bond with a clean price of 130.24 and a coupon rate of 6%.
"""
maturity = "2025-10-25"
date = "2021-03-05"
coupon_rate = 0.06
clean_price = 130.24 # The price is expressed in percentage of the face value.

last_coupon_date = "2020-10-25"
next_coupon_date = "2021-10-25"

total_period = (pd.to_datetime(next_coupon_date) - pd.to_datetime(last_coupon_date)).days
accrued_period = (pd.to_datetime(date) - pd.to_datetime(last_coupon_date)).days

accrued_interest = (accrued_period / total_period) * coupon_rate
print(f'The year is {total_period} days and the accrued period is {accrued_period} days.')

bond_price = clean_price + accrued_interest * 100 # We need to express the accrued interest in percentage of the face value as well.
print(f'The bond price is {bond_price}.')

The year is 365 days and the accrued period is 131 days.
The bond price is 132.39342465753427.


# ***Bond Futures***

In [15]:
"""
Bond Futures are contracts on a fictive underlying. 
When the futures contract expires, the seller of the contract must get a real bond to deliver.
To facilitate the delivery, the seller can deliver one of the bonds that are eligible for delivery.
"""

eligible_bonds = pd.DataFrame({
    "Bond": ["Bund1", "Bund2", "Bund3", "Bund4"],
    "Coupon rate": [0.0175, 0.015, 0.015, 0.015],
    "Maturity": ["2030-07-04", "2030-09-04", "2031-02-15", "2031-05-15"],
    "Conversion factor": [0.715427, 0.694342, 0.682731, 0.676665],
})

#? The futures underlying is a bond with a 6% coupon rate and a maturity of 10 years.
#? As such a bond doesn't exist, we apply a conversion factor to the eligible bonds to make them equivalent to the underlying.

nominal = 100000 # The underlying face value is 100,000$
futures_price = 100 # The futures price is expressed in percentage of the face value.

date = "2021-09-23"
last_coupon_date = pd.to_datetime(eligible_bonds["Maturity"][0]) - pd.DateOffset(years=9)
next_coupon_date = pd.to_datetime(eligible_bonds["Maturity"][0]) - pd.DateOffset(years=8)

total_period = (next_coupon_date - last_coupon_date).days
accrued_period = (pd.to_datetime(date) - last_coupon_date).days

accrued_interest = ((accrued_period / total_period) * eligible_bonds["Coupon rate"][0]) * futures_price
conversion_factor = eligible_bonds["Conversion factor"][0] * futures_price
futures_bond_price = nominal * ((conversion_factor + accrued_interest) / futures_price)
print(f'The bond price is {futures_bond_price}.')

The bond price is 71931.05616438357.


---
# ***Problems & Exercices***

In [16]:
"""
6.11
Determining the accrued interest for a Treasury bond and a Corporate bond on 2020-08-09.
"""
coupon_rate = 0.07 / 2
last_coupon_date = "2020-07-07"
next_coupon_date = "2021-01-07"
date = "2020-08-09"
nominal = 100

# ======= For Treasury bonds (Exact/Exact) =======
total_period = (pd.to_datetime(next_coupon_date) - pd.to_datetime(last_coupon_date)).days
accrued_period = (pd.to_datetime(date) - pd.to_datetime(last_coupon_date)).days

accrued_interest = (accrued_period / total_period) * (coupon_rate * nominal)
print(f'The accrued interest for a Treasury bond is {accrued_interest}.')

# ======= For Corporate bonds (30/360) =======
total_period = days_30_360(last_coupon_date, next_coupon_date)
accrued_period = days_30_360(last_coupon_date, date)

accrued_interest = (accrued_period / total_period) * (coupon_rate * nominal)
print(f'The accrued interest for a Corporate bond is {accrued_interest}.')

The accrued interest for a Treasury bond is 0.6277173913043479.
The accrued interest for a Corporate bond is 0.6222222222222223.


In [19]:
"""
6.12
Determining the bond price for a bond with a clean price of 102-07 and a coupon rate of 6% on 2018-01-09.
"""
date = "2018-01-09"
coupon_rate = 0.06
maturity = "2030-10-12"
quotation = "102-07"

def quotation_to_price(quotation):
    """
    Convert a quotation to a price.
    
    Args:
        quotation (str): The quotation.
    
    Returns:
        float: The price.
    """
    price = float(quotation.split("-")[0]) + float(quotation.split("-")[1]) / 32
    return price

clean_price = quotation_to_price(quotation)
last_coupon_date = pd.to_datetime(maturity) - pd.DateOffset(years=13)
next_coupon_date = pd.to_datetime(maturity) - pd.DateOffset(years=12)

total_period = (next_coupon_date - last_coupon_date).days
accrued_period = (pd.to_datetime(date) - last_coupon_date).days
print(f"The semester is {total_period} days and the accrued period is {accrued_period} days.")
accrued_interest = (accrued_period / total_period) * (coupon_rate * 100)

bond_price = clean_price + accrued_interest
print(f'The bond price is {bond_price}.')

The semester is 365 days and the accrued period is 89 days.
The bond price is 103.68176369863014.


In [21]:
"""
6.13
Computing the P/L for a long position on a bond future.
"""
old_quotation = 96.76
new_quotation = 96.82
position = 2 # long
value_per_bp = 25 # The value per basis point is 25$ per contract, it is a standard value.

bp_return = (new_quotation - old_quotation) * 100 # The return is expressed in basis points.
pl = bp_return * position * value_per_bp

print(f"The P/L of the positon is {pl}.")

The P/L of the positon is 299.9999999999403.


In [31]:
"""
6.14
Computing the Zero Coupon rate given the forward Rate of a Eurodollar futures contract.
"""
LIBOR = 0.03 # 350 days
forward_rate = 0.032 
forward_maturity = 350

# The forward give us the rate we'll have in 350 days. Before that we have the LIBOR rate.
zc_maturity = 440
zc_rate = ((LIBOR * forward_maturity + forward_rate * (zc_maturity - forward_maturity)) / zc_maturity)
print(f"The Zero Coupon rate is {zc_rate}.")

The Zero Coupon rate is 0.030409090909090906.


In [32]:
"""
6.15
Hedging a bond portfolio against interest rate risk.
"""
portfolio_value = 6e6
portfolio_duration_6months = 8.2 # The duration of the portfolio wil be 8.2 years in 6 months.
tbond_futures_quotation = quotation_to_price("108-15") # The quotation of the Treasury bond futures with 6 months to maturity.
cheapest_to_deliver_bond_duration = 7.6
futures_nominal = 1000

# We can Hedge the portfolio by selling the Treasury bond futures.
futures_contract_value = tbond_futures_quotation * futures_nominal
nb_futures_to_short = (portfolio_value / futures_contract_value) * (portfolio_duration_6months / cheapest_to_deliver_bond_duration)
print(f"The number of futures to short is {nb_futures_to_short}.")

The number of futures to short is 59.68248191784561.


In [37]:
"""
6.16
Computing the interest rate of a bond in base Exact/365.
"""
nominal = 100
maturity = 90 # The bond has a maturity of 90 days.
quotation = 10

interest_value = quotation / (360 / maturity)
bond_value = nominal - interest_value

continuous_bond_rate = (365 / maturity) * np.log(nominal / bond_value)
print(f"The interest rate of the bond is {continuous_bond_rate}.")

The interest rate of the bond is 0.10267777682517525.


In [39]:
"""
6.18
Computing the cheapest to deliver bond.
"""
tbond_futures_quotation = quotation_to_price("101-12")
data = pd.DataFrame({
    "Bond": ["Bund1", "Bund2", "Bund3", "Bund4"],
    "Quotation": ["125-05", "142-15", "115-31", "144-02"],
    "Concordance": [1.1231, 1.3792, 1.1149, 1.4026]
})

# We want to minimise Price - Concordance * Futures Quotation
data["Price"] = data["Quotation"].apply(quotation_to_price)
data['Delivery Price'] = data["Price"] - data["Concordance"] * tbond_futures_quotation
cheapest_to_deliver = data.loc[data['Delivery Price'].idxmin()]
print(f"The cheapest to deliver bond is {cheapest_to_deliver['Bond']} for {cheapest_to_deliver['Delivery Price']}.")

The cheapest to deliver bond is Bund4 for 1.8739249999999856.


In [51]:
"""
6.21
Computing the future price of a SOFR 3 months.
"""
SOFR_6months = 0.075
SOFR_9months = 0.08
futures_maturity = 6/12 # The futures deliver the sofr contract in 6 months.
underlying_maturity = 3/12 # The underlying SOFR contract has a maturity of 3 months.

forward_3months_rate = SOFR_9months * (futures_maturity + underlying_maturity) - SOFR_6months * futures_maturity
forward_annual_rate = forward_3months_rate * 4

forward_real_annual_rate = ((1 + forward_annual_rate / 4) ** 4 - 1) * 360/365
forward_price = 100 - forward_real_annual_rate * 100
print(f"The future price is {forward_price}.")

The future price is 90.81917951669523.
